# advent of code 2024 - [day 10](https://adventofcode.com/2024/day/10)

### Parsing

In [33]:
def gen_lists(file='input.txt'):
    """Generates tuples of integers"""
    file = open(file, 'r')
    for ix, line in enumerate(file):
        for jx, c in enumerate(line.strip()):
            yield ix, jx, int(c)

In [34]:
filename = "input.txt"
#filename = "test.txt"

## Graph Solution

In [35]:
import os

NEO4J_URI = os.environ['NEO4J_URI']
NEO4J_USERNAME = os.environ['NEO4J_USERNAME']
NEO4J_PASSWORD = os.environ['NEO4J_PASSWORD']

from graphdatascience import GraphDataScience
gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# Check the installed GDS version on the server
print(gds.version())
assert gds.version()

from neo4j import GraphDatabase
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

2.13.0


In [36]:
# cleaning
def clean():
    queries = [
    "CALL apoc.schema.assert({},{});",
    """MATCH (n)
    CALL {WITH n DETACH DELETE n}
    IN TRANSACTIONS OF 1000 ROWS;""",
    """CALL gds.graph.list()
    YIELD graphName
    WITH graphName AS g
    CALL (g) {CALL gds.graph.drop(g) YIELD graphName RETURN graphName}
    WITH graphName RETURN graphName;"""]

    for q in queries:
        gds.run_cypher(q, {})

### Ingestion


In [37]:
def ingest(filename):

    tiles = [{'row':ix, 'col':jx, 'val':c} for ix, jx, c in list(gen_lists(filename))]
    
    clean()

    gds.run_cypher('CREATE INDEX tile_col_row IF NOT EXISTS FOR (t:Tile) ON (t.col, t.row)')
    gds.run_cypher('CREATE INDEX tile_val IF NOT EXISTS FOR (t:Tile) ON (t.val)')
    
    query_ingest = """
    UNWIND $tiles AS tile
    CREATE (:Tile {row:tile.row, col:tile.col, val:tile.val} )
    """

    gds.run_cypher(query_ingest, {"tiles":tiles})

## part 1

In [38]:
part1_build_queries = ["""MATCH (t:Tile)
WITH t.row AS row_num, t ORDER BY t.col
WITH row_num, collect(t) AS row
CALL apoc.nodes.link(row, 'EAST')""",
"""MATCH (t:Tile)
WITH t.col AS col_num, t ORDER BY t.row
WITH col_num, collect(t) AS col
CALL apoc.nodes.link(col, 'SOUTH')""",
"""
MATCH (a)-[:SOUTH|EAST]-(b)
WHERE a.val + 1 = b.val
MERGE (a)-[:POSSIBLE_MOVE]->(b)
"""]

In [39]:
ingest(filename)
for q in part1_build_queries:
    gds.run_cypher(q)

gds.run_cypher("""MATCH (trail_head:Tile {val:0})
CALL (trail_head) {
  MATCH SHORTEST 1 (trail_head)-[:POSSIBLE_MOVE]->*(top {val: 9})
  RETURN count(DISTINCT top) AS score
} IN CONCURRENT TRANSACTIONS OF 1 ROWS
RETURN sum(score) AS part1""")

,part1
0,746


## Part 2

In [40]:
gds.run_cypher("""MATCH (trail_head:Tile {val:0})
CALL (trail_head) {
  MATCH (trail_head)-[:POSSIBLE_MOVE]->*(top {val: 9})
  RETURN count(top) AS score
} IN CONCURRENT TRANSACTIONS OF 1 ROWS
RETURN sum(score) AS part2""")

,part2
0,1541
